In [7]:
from chainer.datasets import mnist
from chainer.datasets import TupleDataset
from chainer.datasets import split_dataset_random
import numpy as np

train_load = np.load("MHT-train.npy")
train = TupleDataset(train_load[:,0],train_load[:,1])
test_load = np.load("MHT-test.npy")
test = TupleDataset(test_load[:,0],test_load[:,1])
# train: 450 x (framelen x 26 , framelen)

In [8]:
import chainer
import numpy as np

gpu_id = 0

xp = np
if gpu_id >= 0:
    xp = chainer.cuda.cupy

from chainer import iterators

batchsize = 100

train_iter = iterators.SerialIterator(train, batchsize)

In [9]:
import random
import numpy
import chainer

def reset_seed(seed=0):
    random.seed(seed)
    numpy.random.seed(seed)
    if chainer.cuda.available:
        chainer.cuda.cupy.random.seed(seed)

reset_seed(0)

In [10]:
import chainer
import chainer.links as L
import chainer.functions as F

class RNN(chainer.Chain):

    def __init__(self, n_lstm_layers=1, n_mid_units=100, n_out=40, dropout=0.2):
        super(RNN, self).__init__()

        # パラメータを持つ層の登録
        with self.init_scope():
            # self.l1 = L.Linear(None, n_mid_units)
            self.l2 = L.NStepLSTM(n_lstm_layers, 26, n_mid_units, dropout)
            self.l3 = L.Linear(n_mid_units, n_out)

    def __call__(self, x):
        # データを受け取った際のforward計算を書く
        # h1 = F.relu(self.l1(x))
        hy, cy, ys = self.l2(None, None, x)
        h2 = F.concat(ys, axis=0)
        return self.l3(h2)

net = RNN(n_lstm_layers=2, n_mid_units=200)

if gpu_id >= 0:
    net.to_gpu(gpu_id)

In [11]:
from chainer import optimizers

optimizer = optimizers.SGD(lr=0.1).setup(net)

In [12]:
import numpy as np
from chainer import Variable
from chainer.backends import cuda
from chainer.cuda import to_cpu
from chainer.dataset import concat_examples

max_epoch = 500
bprop_len = 200

batchno = 0
while train_iter.epoch < max_epoch:

    # ---------- 学習の1イテレーション ----------
    train_batch = train_iter.next() # batchsize x (framelen x 26 , framelen)
    #batchno += 1
    #print("batch no {}".format(batchno))
    loss = 0
    
    xs = []
    ts = []
    for X, t in train_batch:
        if gpu_id >= 0:
            X = cuda.to_gpu(X, gpu_id)
            t = cuda.to_gpu(t, gpu_id)
        t = t[:,np.newaxis]
        xs.append(chainer.Variable(X.astype(dtype=xp.float32)))
        ts.append(chainer.Variable(t))

    # 予測値の計算
    ys = net(xs)

    # ロスの計算
    ts = F.concat(ts, axis=0)
    ts = ts.reshape(ts.shape[0],)
    loss = F.softmax_cross_entropy(ys, ts)

    # 勾配の計算
    net.cleargrads()
    loss.backward()

    # バッチ単位で古い記憶を削除し、計算コストを削減する。
    loss.unchain_backward()

    # パラメータの更新
    optimizer.update()
    # --------------- ここまで ----------------

    # 1エポック終了ごとにValidationデータに対する予測精度を測って、
    # モデルの汎化性能が向上していることをチェックしよう
    if train_iter.is_new_epoch:  # 1 epochが終わったら
        batchno = 0

        # ロスの表示
        print('epoch:{:02d} train_loss:{:.04f} '.format(
            train_iter.epoch, float(to_cpu(loss.data))), end='')

        valid_losses = []
        valid_accuracies = []

        xs = []
        ts = []
        for X, t in test:
            if gpu_id >= 0:
                X = cuda.to_gpu(X, gpu_id)
                t = cuda.to_gpu(t, gpu_id)
            t = t[:,np.newaxis]
            xs.append(chainer.Variable(X.astype(dtype=xp.float32)))
            ts.append(chainer.Variable(t))
        
        # Validationデータをforward
        with chainer.using_config('train', False), \
                chainer.using_config('enable_backprop', False):
            ys = net(xs)

        ts = F.concat(ts, axis=0)
        ts = ts.reshape(ts.shape[0],)
        # ロスを計算
        loss_valid = F.softmax_cross_entropy(ys, ts)
        valid_losses.append(to_cpu(loss_valid.array))

        # 精度を計算
        accuracy = F.accuracy(ys, ts)
        accuracy.to_cpu()
        valid_accuracies.append(accuracy.array)

        print('val_loss:{:.04f} val_accuracy:{:.04f}'.format(
            np.mean(valid_losses), np.mean(valid_accuracies)))

epoch:01 train_loss:3.5112 val_loss:3.4562 val_accuracy:0.2560
epoch:02 train_loss:3.3043 val_loss:3.2585 val_accuracy:0.2743
epoch:03 train_loss:3.0910 val_loss:3.0419 val_accuracy:0.2913
epoch:04 train_loss:2.9280 val_loss:2.8969 val_accuracy:0.3151
epoch:05 train_loss:2.7899 val_loss:2.7402 val_accuracy:0.3436
epoch:06 train_loss:2.6744 val_loss:2.6290 val_accuracy:0.3616
epoch:07 train_loss:2.5263 val_loss:2.5098 val_accuracy:0.3783
epoch:08 train_loss:2.4909 val_loss:2.4287 val_accuracy:0.3873
epoch:09 train_loss:2.3897 val_loss:2.3445 val_accuracy:0.3969
epoch:10 train_loss:2.3499 val_loss:2.2869 val_accuracy:0.4037
epoch:11 train_loss:2.2639 val_loss:2.2243 val_accuracy:0.4099
epoch:12 train_loss:2.2266 val_loss:2.1807 val_accuracy:0.4155
epoch:13 train_loss:2.1619 val_loss:2.1309 val_accuracy:0.4213
epoch:14 train_loss:2.1249 val_loss:2.0940 val_accuracy:0.4283
epoch:15 train_loss:2.0828 val_loss:2.0513 val_accuracy:0.4365
epoch:16 train_loss:2.0386 val_loss:2.0196 val_accuracy

epoch:131 train_loss:0.8320 val_loss:0.8195 val_accuracy:0.7639
epoch:132 train_loss:0.8222 val_loss:0.8139 val_accuracy:0.7658
epoch:133 train_loss:0.8061 val_loss:0.8106 val_accuracy:0.7662
epoch:134 train_loss:0.7943 val_loss:0.8120 val_accuracy:0.7658
epoch:135 train_loss:0.8095 val_loss:0.8077 val_accuracy:0.7672
epoch:136 train_loss:0.8031 val_loss:0.8409 val_accuracy:0.7599
epoch:137 train_loss:0.8953 val_loss:1.0858 val_accuracy:0.6677
epoch:138 train_loss:0.8127 val_loss:0.8218 val_accuracy:0.7639
epoch:139 train_loss:0.7888 val_loss:0.8055 val_accuracy:0.7687
epoch:140 train_loss:0.8016 val_loss:0.8016 val_accuracy:0.7698
epoch:141 train_loss:0.7994 val_loss:0.7982 val_accuracy:0.7705
epoch:142 train_loss:0.7610 val_loss:0.7951 val_accuracy:0.7709
epoch:143 train_loss:0.7854 val_loss:0.7913 val_accuracy:0.7731
epoch:144 train_loss:0.7905 val_loss:0.7928 val_accuracy:0.7716
epoch:145 train_loss:0.7721 val_loss:0.7928 val_accuracy:0.7715
epoch:146 train_loss:0.7645 val_loss:0.7

epoch:260 train_loss:0.5569 val_loss:0.6273 val_accuracy:0.8167
epoch:261 train_loss:0.5681 val_loss:0.6277 val_accuracy:0.8168
epoch:262 train_loss:0.5577 val_loss:0.6276 val_accuracy:0.8167
epoch:263 train_loss:0.5613 val_loss:0.6260 val_accuracy:0.8162
epoch:264 train_loss:0.5813 val_loss:0.6456 val_accuracy:0.8103
epoch:265 train_loss:0.6893 val_loss:0.9219 val_accuracy:0.7307
epoch:266 train_loss:0.5696 val_loss:0.6227 val_accuracy:0.8150
epoch:267 train_loss:0.5746 val_loss:0.6150 val_accuracy:0.8183
epoch:268 train_loss:0.5689 val_loss:0.6149 val_accuracy:0.8186
epoch:269 train_loss:0.5603 val_loss:0.6145 val_accuracy:0.8186
epoch:270 train_loss:0.5584 val_loss:0.6139 val_accuracy:0.8196
epoch:271 train_loss:0.5380 val_loss:0.6123 val_accuracy:0.8199
epoch:272 train_loss:0.5638 val_loss:0.6141 val_accuracy:0.8204
epoch:273 train_loss:0.5621 val_loss:0.6146 val_accuracy:0.8208
epoch:274 train_loss:0.5545 val_loss:0.6114 val_accuracy:0.8190
epoch:275 train_loss:0.5551 val_loss:0.6

epoch:389 train_loss:0.4511 val_loss:0.5439 val_accuracy:0.8371
epoch:390 train_loss:0.4414 val_loss:0.5382 val_accuracy:0.8391
epoch:391 train_loss:0.4601 val_loss:0.5405 val_accuracy:0.8385
epoch:392 train_loss:0.4425 val_loss:0.5417 val_accuracy:0.8387
epoch:393 train_loss:0.4678 val_loss:0.5390 val_accuracy:0.8390
epoch:394 train_loss:0.4602 val_loss:0.5411 val_accuracy:0.8380
epoch:395 train_loss:0.4374 val_loss:0.5386 val_accuracy:0.8393
epoch:396 train_loss:0.4482 val_loss:0.5393 val_accuracy:0.8388
epoch:397 train_loss:0.4244 val_loss:0.5476 val_accuracy:0.8363
epoch:398 train_loss:0.4460 val_loss:0.5353 val_accuracy:0.8402
epoch:399 train_loss:0.4437 val_loss:0.5347 val_accuracy:0.8377
epoch:400 train_loss:0.4414 val_loss:0.5394 val_accuracy:0.8395
epoch:401 train_loss:0.4383 val_loss:0.5363 val_accuracy:0.8394
epoch:402 train_loss:0.4399 val_loss:0.5361 val_accuracy:0.8398
epoch:403 train_loss:0.4371 val_loss:0.5394 val_accuracy:0.8383
epoch:404 train_loss:0.4456 val_loss:0.5

In [13]:
if gpu_id >= 0:
    net.to_gpu(gpu_id)

# 1つ目のテストデータを取り出します
X, t = test[0]  #  tは使わない

# print('ミニバッチの形にしたあと：', x.shape)

xs = []
if gpu_id >= 0:
    X = cuda.to_gpu(X, gpu_id)
xs.append(chainer.Variable(X.astype(dtype=xp.float32)))

# Validationデータをforward
with chainer.using_config('train', False), \
        chainer.using_config('enable_backprop', False):
    y_test = net(xs)

# Variable形式で出てくるので中身を取り出す
y_test = y_test.array

# 結果をCPUに送る
y_test = to_cpu(y_test)

# 予測確率の最大値のインデックスを見る
pred_label = y_test.argmax(axis=1)

In [14]:
from sklearn.preprocessing import LabelEncoder
with open('../phones') as f:
    phones = f.read().splitlines()
le = LabelEncoder()
le.fit(phones)
le.inverse_transform(t)

/var/tmp/hiroki/conda/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil',
       'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil',
       'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch',
       'ch', 'ch', 'ch', 'ch', 'ch', 'i', 'i', 'i', 'i', 'i', 'i', 'i',
       'i', 'i', 'i', 'i', 's', 's', 's', 's', 's', 's', 's', 's', 's',
       's', 'a', 'a', 'a', 'a', 'a', 'a', 'n', 'n', 'n', 'n', 'a', 'a',
       'a', 'a', 'a', 'a', 'a', 'a', 'a', 'u', 'u', 'u', 'u', 'u', 'u',
       'u', 'u', 'u', 'n', 'n', 'n', 'n', 'n', 'a', 'a', 'a', 'a', 'a',
       'a', 'a', 'a', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'i', 'i',
       'i', 'i', 'i', 'i', 'i', 'y', 'y', 'y', 'y', 'y', 'y', 'y', 'a',
       'a', 'a', 'a', 'a', 'a', 'a', 'a', 'n', 'n', 'n', 'n', 'i', 'i',
       'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i',
       'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau',
       'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau',

In [15]:
le.inverse_transform(pred_label)

/var/tmp/hiroki/conda/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil',
       'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil',
       'ch', 'ch', 't', 't', 't', 't', 't', 'ch', 'ch', 'ch', 'ch', 'ch',
       'ch', 'ch', 'ch', 'ch', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i',
       'i', 'i', 'i', 's', 's', 's', 's', 's', 's', 's', 's', 's', 'e',
       'a', 'a', 'a', 'a', 'a', 'n', 'n', 'n', 'n', 'n', 'a', 'a', 'a',
       'a', 'a', 'a', 'a', 'a', 'r', 'u', 'u', 'u', 'u', 'u', 'u', 'u',
       'u', 'N', 'N', 'n', 'n', 'n', 'n', 'a', 'a', 'a', 'a', 'a', 'a',
       'a', 'a', 'g', 'g', 'g', 'n', 'n', 'n', 'g', 'i', 'i', 'i', 'i',
       'i', 'i', 'i', 'i', 'i', 'i', 'i', 'y', 'y', 'y', 'a', 'a', 'a',
       'a', 'a', 'a', 'a', 'r', 'r', 'n', 'n', 'n', 'n', 'i', 'i', 'i',
       'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'pau',
       'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau',
       'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau